# Collecte de données d’emploi via une API

* Ce projet vise à illustrer un flux complet de collecte de données à partir d’une API, suivi d’un traitement simple et d’une exportation vers Excel.
* Pour simuler une source externe, j’exécute localement une petite API Flask qui expose un ensemble d’offres d’emploi. Cela me permet de travailler dans des conditions proches d’un environnement réel où les données proviennent d’un service web.

## 1. Contexte et approche

Dans un pipeline d’analyse moderne, les données ne proviennent pas toujours de fichiers statiques : beaucoup transitent par des APIs.
Ici, je reproduis ce type de scénario :

Une API Flask locale agit comme source de données.

Les données sont exposées via plusieurs endpoints.

Le notebook interroge l’API, extrait les données pertinentes puis les stocke dans un fichier Excel.

Ce workflow illustre comment un analyste peut interagir avec une API, automatiser des requêtes et préparer des résultats destinés à une analyse ultérieure.

## 2. API simulée : rôle et fonctionnement

L’API charge un dataset JSON contenant des offres d’emploi (compétences, localisation, rôle, industrie, etc.). Elle expose notamment :
* GET /data/all — renvoie toutes les offres.
* GET /data?... — renvoie les offres filtrées en fonction des paramètres fournis.

Quelques exemples de paramètres supportés :

* Job Title
* Key Skills
* Location
* Industry
* Role

Le backend utilise de simples expressions régulières pour identifier les compétences ou les villes recherchées.
L’objectif ici n’est pas de créer un backend complexe, mais d’avoir une source suffisamment réaliste pour l’analyse.

## 3. Lancement de l’API
Avant d’interroger les données, je démarre l’API Flask localement.
Une fois en exécution, les endpoints deviennent accessibles :

In [ ]:
api_url="http://127.0.0.1:5000/data"

## 4. Objectif de la collecte

### Deux axes d’analyse :

#### A. Nombre de postes disponibles par ville 
* Los Angeles
* New York
* San Francisco
* Washington DC
* Seattle
* Austin
* Detroit

#### B. Nombre de postes par compétence
Technologies observées :
* C, C#, C++, Java, JavaScript, Python, Scala
* Oracle, SQL Server, MySQL Server, PostgreSQL, MongoDB

Ces deux volets permettent de comparer la demande entre régions et compétences.

## 5. Fonctions d’interrogation de l’API

#### A. Nombre de postes par technologie

In [ ]:
#chargement des bib necessaire 
import pandas as pd
import json
import requests 

In [ ]:
def get_number_of_jobs_T(technology):
    number_of_jobs=None
    par={"Key Skills":technology}
    try:
        response=requests.get(api_url,params=par)
        if response.status_code==200:
            number_of_jobs=len(response.json())
    except requests.exceptions.RequestException as e:
        print('erreur : ',e)
    return technology,number_of_jobs

#### B. Nombre de postes par localisation

In [ ]:
def get_number_of_jobs_L(location):    
    par={"Location":location}
    try:
        response=requests.get(api_url,params=par)
        if response.status_code==200:
            number_of_jobs=len(response.json())
    except requests.exceptions.RequestException as e:
        print('erreur : ',e)
    return location,number_of_jobs

In [ ]:
# test de la fonction
a,b=get_number_of_jobs_T("Python")
print(a,b)

Python 1173


In [ ]:
# test de la fonction
print(get_number_of_jobs_L("Los Angeles"))


('Los Angeles', 640)


## 6. Export des résultats

Les résultats sont regroupés dans un fichier Excel :
* Onglet jobs locations → nombre de postes par ville
* Onglet langages jobs → nombre de postes par compétence


In [ ]:
#
locations=["Los Angeles","New York","San Francisco","Washington DC","Seattle","Austin","Detroit"]
fichier ="job-postings.xlsx"


In [ ]:
# your code goes here
from openpyxl import Workbook,load_workbook
import os 
path=os.path.join(os.getcwd(),fichier)

if os.path.exists(fichier):
    wb=load_workbook(fichier)
    ws=wb.create_sheet("jobs locations")
else :
    wb=Workbook()
    wb.active.title="jobs locations"
    ws=wb.active

ws.append(("Location","job post"))
for x in locations:
    ws.append(tuple(get_number_of_jobs_L(x)))

wb.save(fichier)
wb.close()


In [ ]:
# your code goes here
langages=["C","C#","C++","Java","JavaScript","Python","Scala","Oracle","SQL Server","MySQL Server","PostgreSQL","MongoDB"]

In [ ]:
if os.path.exists(fichier):
    wb=load_workbook(fichier)
    ws=wb.create_sheet("langages jobs")
else :
    wb=Workbook()
    ws=wb.active
    ws.title="luguages jobs"

ws.append(("langage","nbr post"))
for x in langages:
    ws.append(tuple(get_number_of_jobs_T(x)))
wb.save(fichier)
wb.close()